In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import tkinter as tk
from tkinter import ttk, messagebox
from datetime import datetime, timedelta
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import warnings
import os
from tkcalendar import DateEntry
warnings.filterwarnings('ignore')

class SteelPriceForecastApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Оптимизация закупок арматуры")
        self.root.geometry("1200x800")
        
        try:
            self.load_data()
            self.prepare_data()
            self.create_widgets()
            self.plot_history()
        except Exception as e:
            messagebox.showerror("Ошибка", f"Ошибка инициализации: {str(e)}")
            self.root.destroy()
    
    def load_data(self):
        """Загрузка данных из обоих файлов"""
        file_path = "C:\\Users\\Karina\\Desktop\\Новая папка\\обработанные данные.csv"
        df2_path = "C:\\Users\\Karina\\Desktop\\Новая папка\\train_clean.csv"
        
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Файл не найден: {file_path}")
        if not os.path.exists(df2_path):
            raise FileNotFoundError(f"Файл не найден: {df2_path}")
        
        try:
            # Загружаем оба файла
            self.df = pd.read_csv(file_path, parse_dates=['Date'])
            self.df2 = pd.read_csv(df2_path, parse_dates=['dt'])
            
            if len(self.df) < 10 or len(self.df2) < 10:
                raise ValueError("Недостаточно данных для анализа (минимум 10 записей)")
                
            # Проверяем необходимые колонки
            if 'Date' not in self.df.columns or 'Цена на арматуру_x_stationary' not in self.df.columns:
                raise ValueError("Основной файл не содержит нужных колонок")
            if 'dt' not in self.df2.columns or 'Цена на арматуру' not in self.df2.columns:
                raise ValueError("Файл с оригинальными ценами не содержит нужных колонок")
                
            # Сортируем и устанавливаем индекс
            self.df = self.df.sort_values(by='Date').set_index('Date')
            self.df2 = self.df2.sort_values(by='dt').set_index('dt')
            
            # Сохраняем оба набора цен
            self.stationary_prices = self.df['Цена на арматуру_x_stationary'].copy()
            self.original_prices = self.df2['Цена на арматуру'].copy()
            
            # Для анализа используем стационарные данные
            self.price_series = self.stationary_prices.copy()
            
        except Exception as e:
            raise ValueError(f"Ошибка загрузки данных: {str(e)}")
    
    def prepare_data(self):
        """Подготовка данных для анализа"""
        try:
            # Очистка и интерполяция стационарных данных
            self.price_series = self.price_series.replace([np.inf, -np.inf], np.nan)
            self.price_series = self.price_series.interpolate(method='time').dropna()
            
            if len(self.price_series) < 10:
                raise ValueError("После очистки осталось слишком мало данных")
            
            # Проверка стационарности
            try:
                adf_result = adfuller(self.price_series.dropna())
                if adf_result[1] > 0.05:
                    self.price_series = self.price_series.diff().dropna()
            except:
                pass
            
            # Текущая цена берется из оригинальных данных
            self.current_price = self.original_prices.iloc[-1]
            self.available_dates = self.original_prices.index.date.tolist()
            
            # Определяем частоту данных (в днях)
            self.data_freq = (self.original_prices.index[-1] - self.original_prices.index[-2]).days
            
        except Exception as e:
            raise ValueError(f"Ошибка подготовки данных: {str(e)}")
    
    def create_widgets(self):
        """Создание интерфейса"""
        try:
            main_frame = ttk.Frame(self.root)
            main_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)
            
            header = ttk.Label(main_frame, text="Оптимальное время закупки арматуры", 
                             font=('Arial', 16, 'bold'))
            header.pack(pady=15)
            
            control_frame = ttk.Frame(main_frame)
            control_frame.pack(fill=tk.X, pady=15)
            
            ttk.Label(control_frame, text="Выберите дату анализа:", font=('Arial', 11)).pack(side=tk.LEFT, padx=5)
            
            self.cal = DateEntry(
                control_frame,
                width=12,
                background='darkblue',
                foreground='white',
                borderwidth=2,
                date_pattern='yyyy-mm-dd',
                mindate=self.available_dates[0],
                maxdate=self.available_dates[-1]
            )
            self.cal.set_date(self.available_dates[-1])
            self.cal.pack(side=tk.LEFT, padx=5)
            
            analyze_btn = ttk.Button(control_frame, text="Анализировать", 
                                   command=self.safe_analyze_prices,
                                   style='Large.TButton')
            analyze_btn.pack(side=tk.LEFT, padx=15)
            
            style = ttk.Style()
            style.configure('Large.TButton', font=('Arial', 11), padding=6)
            
            self.result_frame = ttk.Frame(main_frame)
            self.result_frame.pack(fill=tk.X, pady=20, padx=10)
            
            self.result_label = ttk.Label(
                self.result_frame, 
                text="Выберите дату и нажмите 'Анализировать'", 
                font=('Arial', 12), 
                wraplength=900,
                justify=tk.LEFT,
                background='#f0f8ff',
                relief=tk.GROOVE,
                padding=(15, 10),
                borderwidth=2
            )
            self.result_label.pack(fill=tk.X)
            
            self.figure = plt.figure(figsize=(12, 6), dpi=100, facecolor='#f8f8f8')
            self.canvas = FigureCanvasTkAgg(self.figure, master=main_frame)
            self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
            
        except Exception as e:
            raise ValueError(f"Ошибка создания интерфейса: {str(e)}")
    
    def safe_analyze_prices(self):
        """Защищенный анализ с обработкой ошибок"""
        try:
            self.analyze_prices()
        except Exception as e:
            messagebox.showerror("Ошибка анализа", f"Не удалось выполнить анализ: {str(e)}")
            self.result_label.config(text="Ошибка при анализе данных")
    
    def plot_history(self):
        """Построение графика стационарных цен из df"""
        try:
            self.figure.clear()
            ax = self.figure.add_subplot(111, facecolor='#f8f8f8')
            
            # Используем стационарные данные для графика
            ax.plot(self.stationary_prices.index, self.stationary_prices, 
                   color='#1f77b4', linewidth=2.5)
            ax.set_title('История цен на арматуру (стационарные данные)', fontsize=14, pad=25)
            ax.set_xlabel('Дата', fontsize=12)
            ax.set_ylabel('Стационарная цена', fontsize=12)
            ax.grid(True, linestyle='--', alpha=0.7)
            ax.set_facecolor('#f8f8f8')
            
            plt.setp(ax.get_xticklabels(), rotation=45, ha='right', fontsize=10)
            plt.setp(ax.get_yticklabels(), fontsize=10)
            plt.tight_layout()
            self.canvas.draw()
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось построить график: {str(e)}")
    
    def find_nearest_date(self, target_date):
        """Находит ближайшую доступную дату к выбранной"""
        target_date = pd.to_datetime(target_date).date()
        dates = pd.to_datetime(self.available_dates).date
        nearest_date = min(dates, key=lambda x: abs(x - target_date))
        return nearest_date
    
    def calculate_forecast_horizon(self, selected_date):
        """Автоматически рассчитывает горизонт прогнозирования на основе выбранной даты"""
        # Определяем, сколько данных осталось после выбранной даты
        last_date = pd.to_datetime(self.available_dates[-1])
        selected_date = pd.to_datetime(selected_date)
        
        # Если выбрана последняя доступная дата, используем эвристику
        if selected_date == last_date:
            return 12  # недель по умолчанию
        
        # Рассчитываем, сколько недель осталось до конца данных
        delta = last_date - selected_date
        weeks_left = delta.days // 7
        
        # Используем минимум между оставшимися неделями и 12 неделями
        forecast_weeks = min(12, max(4, weeks_left))  # Не менее 4 недель
        
        return forecast_weeks
    
    def analyze_prices(self):
        """Анализ с использованием стационарных данных, но вывод оригинальных цен"""
        selected_date = self.cal.get_date()
        
        if selected_date not in self.available_dates:
            nearest_date = self.find_nearest_date(selected_date)
            response = messagebox.askyesno(
                "Дата не найдена", 
                f"Выбранная дата {selected_date.strftime('%d.%m.%Y')} не найдена в данных.\n"
                f"Использовать ближайшую доступную дату {nearest_date.strftime('%d.%m.%Y')}?"
            )
            
            if response:
                selected_date = nearest_date
                self.cal.set_date(selected_date)
            else:
                return
        
        # Для анализа используем стационарные данные
        historical_stationary = self.stationary_prices.loc[:str(selected_date)]
        historical_original = self.original_prices.loc[:str(selected_date)]
        
        if len(historical_stationary) < 10:
            raise ValueError("Недостаточно данных для анализа (минимум 10 точек)")
        
        # Автоматически определяем горизонт прогнозирования
        forecast_weeks = self.calculate_forecast_horizon(selected_date)
        
        # Обучаем модель на стационарных данных
        try:
            model = ARIMA(historical_stationary, order=(1, 1, 1))
            model_fit = model.fit()
        except Exception as e:
            try:
                model = ARIMA(historical_stationary, order=(1, 1, 0))
                model_fit = model.fit()
            except:
                raise ValueError("Не удалось построить модель прогнозирования")
        
        # Получаем прогноз
        try:
            forecast = model_fit.get_forecast(steps=forecast_weeks)
            forecast_values = forecast.predicted_mean
            if len(forecast_values) == 0:
                raise ValueError("Не удалось получить прогноз")
            
            if forecast_values.min() <= 0:
                forecast_values = forecast_values.clip(lower=0.1)
        except:
            raise ValueError("Ошибка при прогнозировании")
        
        optimal_weeks = self.find_optimal_period(forecast_values)
        
        # Для отображения используем оригинальные цены
        current_price = historical_original.iloc[-1]
        
        # Преобразуем прогнозные значения обратно в оригинальный масштаб (если нужно)
        # Здесь просто используем среднее значение прогноза как ориентир
        optimal_price = forecast_values.iloc[:optimal_weeks].mean()
        
        # Форматированный вывод с оригинальными ценами
        result_text = (f"📅 Дата анализа: {selected_date.strftime('%d.%m.%Y')}\n"
                     f"💰 Текущая цена: {current_price:,.2f} руб/т\n"
                     f"📆 Горизонт прогноза: {forecast_weeks} недель\n\n"
                     f"🛒 Рекомендация: закупить в течение {optimal_weeks} недель\n"
                     f"📊 Прогноз среднего изменения цены: {optimal_price:,.2f}\n"
                     f"💵 Текущая цена в рублях: {current_price:,.2f} руб/т")
        
        self.result_label.config(text=result_text)
        self.update_plot(selected_date, historical_stationary, forecast_values, optimal_weeks)
    
    def find_optimal_period(self, forecast):
        """Поиск оптимального периода"""
        try:
            min_avg = float('inf')
            best_weeks = 1
            
            for weeks in range(1, min(12, len(forecast))):
                avg = forecast.iloc[:weeks].mean()
                if avg > 0 and avg < min_avg:
                    min_avg = avg
                    best_weeks = weeks
                    
            return best_weeks if best_weeks > 0 else 1
        except:
            return 1
    
    def update_plot(self, selected_date, history, forecast, optimal_weeks):
        """Обновление графика со стационарными данными"""
        try:
            self.figure.clear()
            ax = self.figure.add_subplot(111, facecolor='#f8f8f8')
            
            selected_date = pd.to_datetime(selected_date)
            
            # Отображаем стационарные данные
            ax.plot(history.index, history, 
                   label='История цен (стационарные данные)', 
                   color='#1f77b4', linewidth=2.5)
            
            # Прогноз
            forecast_dates = pd.date_range(start=selected_date, periods=len(forecast), freq='W')
            ax.plot(forecast_dates, forecast, 
                   label='Прогноз изменения цен', 
                   color='#2ca02c', linewidth=2.5, linestyle='--')
            
            # Выделяем оптимальный период
            if optimal_weeks > 0 and optimal_weeks <= len(forecast_dates):
                ax.axvspan(selected_date, forecast_dates[optimal_weeks-1], 
                          color='#ffd700', alpha=0.3, 
                          label=f'Рекомендуемый период ({optimal_weeks} недель)')
            
            # Линия выбранной даты
            ax.axvline(x=selected_date, color='#d62728', 
                      linestyle=':', linewidth=2, 
                      label='Дата анализа')
            
            # Настройки графика
            ax.set_title('Рекомендации по закупке (на основе стационарных данных)', 
                        fontsize=14, pad=25)
            ax.set_xlabel('Дата', fontsize=12)
            ax.set_ylabel('Стационарная цена', fontsize=12)
            ax.legend(loc='upper left', fontsize=10)
            ax.grid(True, linestyle='--', alpha=0.7)
            
            plt.setp(ax.get_xticklabels(), rotation=45, ha='right', fontsize=10)
            plt.setp(ax.get_yticklabels(), fontsize=10)
            plt.tight_layout()
            self.canvas.draw()
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось обновить график: {str(e)}")

if __name__ == "__main__":
    try:
        root = tk.Tk()
        app = SteelPriceForecastApp(root)
        root.mainloop()
    except Exception as e:
        messagebox.showerror("Критическая ошибка", f"Приложение не может быть запущено: {str(e)}")